# **Introduction to cuML and cuDF**
cuML and cuDF are part of RAPIDS, a suite of GPU-accelerated libraries designed to speed up data science workflows using NVIDIA GPUs.

## **cuDF (GPU DataFrames)**

cuDF is a GPU-accelerated equivalent of pandas for DataFrames.
It enables fast data manipulation by leveraging CUDA and NVIDIA GPUs.
The syntax is similar to pandas, making it easy to adopt.

## **cuML (GPU Machine Learning)**

cuML provides GPU-accelerated implementations of machine learning algorithms.
It supports Logistic Regression, Random Forests, KMeans, PCA, and more.<br>
The API is designed to be similar to Scikit-learn for a smooth transition.


# **Comparing MNIST Classification: CPU vs GPU (cuDF & cuML)**



**Install and Import Dependencies**

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/install_rapids.sh stable
import os
os.environ["PATH"] += ":/usr/local/cuda/bin"


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 586, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 586 (delta 122), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (586/586), 191.99 KiB | 1.03 MiB/s, done.
Resolving deltas: 100% (296/296), done.
bash: rapidsai-csp-utils/colab/install_rapids.sh: No such file or directory


In [ ]:
import cuml
import cudf
print("cuML & cuDF installed successfully")


cuML & cuDF installed successfully


**Load the MNIST Dataset**

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Charger MNIST
mnist = fetch_openml("mnist_784", version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)

# Normalisation et séparation des données
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



**Convert Data to cuDF Format for GPU Processing**

In [ ]:
import cudf

X_train_cudf = cudf.DataFrame(X_train)
X_test_cudf = cudf.DataFrame(X_test)
y_train_cudf = cudf.Series(y_train)
y_test_cudf = cudf.Series(y_test)

print("Data loaded and preprocessed")

Data loaded and preprocessed


**Train on CPU (Scikit-learn)**

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Modèle CPU
rf_cpu = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1)

# Entraînement
start_cpu = time.time()
rf_cpu.fit(X_train, y_train)
end_cpu = time.time()

# Prédiction et évaluation
y_pred_cpu = rf_cpu.predict(X_test)
acc_cpu = accuracy_score(y_test, y_pred_cpu)

print(f"Accuracy (Scikit-learn CPU): {acc_cpu:.4f}")
print(f"Temps d'entraînement (CPU): {end_cpu - start_cpu:.2f} sec")


Accuracy (Scikit-learn CPU): 0.9459
Temps d'entraînement (CPU): 20.23 sec


**Train on GPU (cuML + cuDF)**

In [ ]:
from cuml.ensemble import RandomForestClassifier

# Modèle GPU
rf_gpu = RandomForestClassifier(n_estimators=100, max_depth=10, n_bins=8)

# Entraînement
start_gpu = time.time()
rf_gpu.fit(X_train_cudf, y_train_cudf)
end_gpu = time.time()

# Prédiction et évaluation
y_pred_gpu = rf_gpu.predict(X_test_cudf)
acc_gpu = accuracy_score(y_test_cudf.to_pandas(), y_pred_gpu.to_pandas())

print(f"Accuracy (cuML GPU): {acc_gpu:.4f}")
print(f"Temps d'entraînement (GPU): {end_gpu - start_gpu:.2f} sec")


Accuracy (cuML GPU): 0.9447
Temps d'entraînement (GPU): 4.58 sec


**Compare CPU vs GPU Performance**

In [ ]:
import pandas as pd

results = pd.DataFrame({
    "Méthode": ["Scikit-learn (CPU)", "cuML (GPU)"],
    "Accuracy": [acc_cpu, acc_gpu],
    "Temps d'entraînement (sec)": [end_cpu - start_cpu, end_gpu - start_gpu]
})

print(results)

              Méthode  Accuracy  Temps d'entraînement (sec)
0  Scikit-learn (CPU)  0.945929                   20.226352
1          cuML (GPU)  0.944714                    4.584144



*   cuML on GPU is significantly faster while maintaining nearly identical accuracy.
* For larger datasets, GPU acceleration will provide even more time savings.
* If speed is the priority, cuML is the better choice.
For small datasets, CPU performance may be sufficient.

*   List item

